In [1]:
# Instalação de bibliotecas necessárias

! pip3 install pandas
! pip3 install matplotlib
! pip3 install numpy
! pip3 install sklearn

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no lon

In [2]:
# Importação de bibliotecas de uso geral

import pandas as pd
import sklearn
import statistics 
import matplotlib.pyplot as plt
from decimal import Decimal
import numpy as np

In [3]:
# Leitura do conjunto de dados

data = pd.read_csv('tic-tac-toe.csv')
display(data.describe())
display(data.head())

,tl,tm,tr,ml,mm,mr,bl,bm,br,class
count,958,958,958,958,958,958,958,958,958,958
unique,3,3,3,3,3,3,3,3,3,2
top,x,x,x,x,x,x,x,x,x,positive
freq,418,378,418,378,458,378,418,378,418,626


,tl,tm,tr,ml,mm,mr,bl,bm,br,class
0,x,x,x,x,o,o,x,o,o,positive
1,x,x,x,x,o,o,o,x,o,positive
2,x,x,x,x,o,o,o,o,x,positive
3,x,x,x,x,o,o,o,b,b,positive
4,x,x,x,x,o,o,b,o,b,positive


In [7]:
y = []
for c in data['class']:
    result = 0
    if c == 'positive':
        result = 1
    y.append(result)

In [5]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(['x', 'o', 'b'])
X = data.drop(columns=['class'])
X['tl'] = encoder.transform(X['tl'])
X['tm'] = encoder.transform(X['tm'])
X['tr'] = encoder.transform(X['tr'])
X['ml'] = encoder.transform(X['ml'])
X['mm'] = encoder.transform(X['mm'])
X['mr'] = encoder.transform(X['mr'])
X['bl'] = encoder.transform(X['bl'])
X['bm'] = encoder.transform(X['bm'])
X['br'] = encoder.transform(X['br'])
X.head()

,tl,tm,tr,ml,mm,mr,bl,bm,br
0,2,2,2,2,1,1,2,1,1
1,2,2,2,2,1,1,1,2,1
2,2,2,2,2,1,1,1,1,2
3,2,2,2,2,1,1,1,0,0
4,2,2,2,2,1,1,0,1,0


In [86]:
# Teste de benchmark com os dados usando o AdaBoostClassifier da biblioteca sklearn

from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split

clf = AdaBoostClassifier(n_estimators=100)

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=0)

clf.fit(train_X, train_y)
pred_y = clf.predict(test_X)
errors = 0
for i in range(len(test_y)):
    if test_y[i] != pred_y[i]:
        errors = errors + 1
print('sklearn adaboost error rate: ', errors/len(test_y))

sklearn adaboost error rate:  0.18229166666666666


In [87]:
import numpy as np

def calc_alpha(error_rate):
    return 0.5 * np.log((1 - error_rate) / error_rate)

def calc_weight(w, alpha, misclassified):
    return w * np.exp(alpha * (1 if misclassified else -1))

# X: pandas dataframe
# y: list
# n_iter: integer value
def train_adaboost(X, y, n_iter):
    boost = []
    preditors = []
    total_rows = X.shape[0]
    initial_weight = 1/total_rows
    weights = []
    for col in X:
        col_values = []
        for value in X[col]:
            present = False
            for col_value in col_values:
                if col_value == value:
                    present = True
                    break
            if not present:
                col_values.append(value)
        for col_value in col_values:
            preditors.append({
                "col": col,
                "value": col_value,
                "misc": [],
                "alpha": 0
            })
    for i in range(total_rows):
        weights.append(initial_weight)
    y_index = 0
    for index, x in X.iterrows():
        for p in preditors:
            predicted_value = 0
            if x[p['col']] == p['value']:
                predicted_value = 1
            if predicted_value != y[y_index]:
                p['misc'].append(y_index)
        y_index = y_index+1
    for i in range(n_iter):
        min_err = total_rows+1
        min_err_preditor = {}
        for p in preditors:
            preditor_error_rate = 0
            for misc in p['misc']:
                preditor_error_rate = preditor_error_rate + weights[misc]
            if preditor_error_rate < min_err:
                min_err_preditor = p
                min_err = preditor_error_rate
        alpha = calc_alpha(min_err)
        min_err_preditor["alpha"] = alpha
        boost.append(min_err_preditor)
        weight_sum = 0
        for w_index in range(len(weights)):
            misclassified = False
            for misc in min_err_preditor['misc']:
                if misc == w_index:
                    misclassified = True
                    break
            weights[w_index] = calc_weight(weights[w_index], alpha, misclassified)
            weight_sum = weight_sum + weights[w_index]
        z = 1/weight_sum
        for i in range(len(weights)):
            weights[i] = weights[i] * z
    return boost

def predict_adaboost(boost, X):
    y = []
    for index, x in X.iterrows():
        predicted_value = 0
        for b in boost:
            col = b['col']
            value = b['value']
            alpha = b['alpha']
            x_value = x[col]
            predicted_value = predicted_value + ((1 if x_value == value else -1) * alpha)
        y.append(1 if predicted_value > 0 else 0)
    return y

clf = train_adaboost(train_X, train_y, 100)
pred_y = predict_adaboost(clf, test_X)

errors = 0
for i in range(len(test_y)):
    if test_y[i] != pred_y[i]:
        errors = errors + 1
print('error rate: ', errors/len(test_y))

error rate:  0.4427083333333333


In [ ]:
print('1/16', 1/16)
print('4/16', 4/16)

alpha = 0.5 * np.log(4)
w = 1/10
misc = calc_weight(w, alpha, True)
not_misc = calc_weight(w, alpha, False)
total = (2*misc) + (8*not_misc)
z = 1/total

print('misc', z*misc)
print('not misc', z*not_misc)